### 📄 TRABALHO DE CLASSIFICAÇÃO COM KNN

**Professor:** Romuere Rodrigues Velosos e Silva

**Equipe:**

1. Iago Roberto

2. Francinaldo Barbosa

3. Cristina de Moura

### ETAPA 0: Importação das bibliotecas

In [163]:
# Importações
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, StratifiedKFold, LeaveOneOut
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, cohen_kappa_score

def moldura(texto: str):
    largura = len(texto) + 4
    print("+" + "-" * (largura - 2) + "+")
    print("| " + texto.center(largura - 4) + " |")
    print("+" + "-" * (largura - 2) + "+")

def add_linha(texto: str, largura_total: int = 90):
    texto = texto.strip()
    if len(texto) + 6 >= largura_total:
        largura_total = len(texto) + 10 

    largura_lados = (largura_total - len(texto) - 4) // 2
    linha = "-" * largura_lados

    if (largura_total - len(texto) - 4) % 2 != 0:
        print(f"\n# {linha} {texto} {linha}- #\n")
    else:
        print(f"\n# {linha} {texto} {linha} #\n")

### ETAPA 1: Carregando o dataset

In [164]:
# 1) Carregar os dados

moldura("Carregando os dados")

fp = Path('data/Thyroid_Diff.csv')
if not fp.exists():
    raise FileNotFoundError(f"Arquivo não encontrado: {fp}. Coloque o CSV em data/ ou ajuste o caminho.")

df = pd.read_csv(fp)

print("Primeiras linhas do dataset:")
print(df.head(6).to_string(index=False))

+---------------------+
| Carregando os dados |
+---------------------+
Primeiras linhas do dataset:
 Age Gender Smoking Hx Smoking Hx Radiothreapy Thyroid Function        Physical Examination Adenopathy      Pathology    Focality Risk   T  N  M Stage      Response Recurred
  27      F      No         No              No        Euthyroid  Single nodular goiter-left         No Micropapillary   Uni-Focal  Low T1a N0 M0     I Indeterminate       No
  34      F      No        Yes              No        Euthyroid         Multinodular goiter         No Micropapillary   Uni-Focal  Low T1a N0 M0     I     Excellent       No
  30      F      No         No              No        Euthyroid Single nodular goiter-right         No Micropapillary   Uni-Focal  Low T1a N0 M0     I     Excellent       No
  62      F      No         No              No        Euthyroid Single nodular goiter-right         No Micropapillary   Uni-Focal  Low T1a N0 M0     I     Excellent       No
  62      F      No         N

### ETAPA 2: Convertendo a coluna alvo para inteiro

In [165]:
# 2) Conversão da última coluna (categórica) para inteiros (0/1)

moldura("Convertendo a coluna alvo 'Recurred' para inteiros")

if 'Recurred' not in df.columns:
    # tentar localizar última coluna
    last_col = df.columns[-1]
    df = df.rename(columns={last_col: 'Recurred'})

# Mapeamento simples: No -> 0, Yes -> 1. Ajuste se existirem outros valores.
unique_vals = df['Recurred'].unique().tolist()
print('\nValores únicos na coluna Recurred:', unique_vals)

mapping = {v: (1 if str(v).strip().lower() in ['yes', 'y', 'sim', 's', '1', 'true', 'TRUE'] else 0) for v in unique_vals}
# Aplicar mapeamento (não destrutivo)
df['result'] = df['Recurred'].map(mapping)

# Tabela antes e depois (curta)
before = df[['Recurred']].head(6).rename(columns={'Recurred': 'Recurred (orig)'})
after = df[['result']].head(6).rename(columns={'result': 'Recurred (mapeado)'})
print('\nTabela - antes e depois (primeiras 6 linhas):')
print(pd.concat([before, after], axis=1).to_string(index=False))

+----------------------------------------------------+
| Convertendo a coluna alvo 'Recurred' para inteiros |
+----------------------------------------------------+

Valores únicos na coluna Recurred: ['No', 'Yes']

Tabela - antes e depois (primeiras 6 linhas):
Recurred (orig)  Recurred (mapeado)
             No                   0
             No                   0
             No                   0
             No                   0
             No                   0
             No                   0


### ETAPA 1: Pré-processamento dos dados

In [166]:
# 3) Pré-processamento: tratar datas, valores ausentes e codificar variáveis

moldura("Pré-processamento dos dados")

# Detectar colunas numéricas e categóricas
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
# Excluir a coluna result das features
if 'result' in numeric_cols:
    numeric_cols.remove('result')

categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

if 'Recurred' in categorical_cols:
    categorical_cols.remove('Recurred')

print('\nColunas numéricas detectadas:', numeric_cols)
print('Colunas categóricas detectadas:', categorical_cols)


num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median'))
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, numeric_cols),
    ('cat', cat_pipeline, categorical_cols)
])

X_raw = df.drop(columns=['Recurred', 'result'])
y = df['result'].copy()

X_proc_array = preprocessor.fit_transform(X_raw)

feature_names_num = numeric_cols

try:
    ohe = preprocessor.named_transformers_['cat'].named_steps['onehot']
    cat_feature_names = ohe.get_feature_names_out(categorical_cols).tolist()
except Exception:
    cat_feature_names = []

feature_names = feature_names_num + cat_feature_names


df_processado = pd.DataFrame(X_proc_array, columns=feature_names)

df_processado['result'] = y.values

add_linha("Resumo do conjunto de dados processado")
print(f"  Amostras: {df_processado.shape[0]}")
print(f"  Features: {df_processado.shape[1] - 1}")
print(f"  Features utilizadas: {list(df_processado.columns.drop('result'))}")

# Verificar distribuição de classes e se há dados suficientes
add_linha("Distribuição das classes")
print(df_processado['result'].value_counts(dropna=False))

n_samples = df_processado.shape[0]
if n_samples < 30:
    print('\nAVISO: o conjunto possui menos de 30 amostras — alguns métodos (LOOCV) podem ficar menos informativos ou ruidosos.)')

+-----------------------------+
| Pré-processamento dos dados |
+-----------------------------+

Colunas numéricas detectadas: ['Age']
Colunas categóricas detectadas: ['Gender', 'Smoking', 'Hx Smoking', 'Hx Radiothreapy', 'Thyroid Function', 'Physical Examination', 'Adenopathy', 'Pathology', 'Focality', 'Risk', 'T', 'N', 'M', 'Stage', 'Response']

# ------------------------ Resumo do conjunto de dados processado ------------------------ #

  Amostras: 383
  Features: 55
  Features utilizadas: ['Age', 'Gender_F', 'Gender_M', 'Smoking_No', 'Smoking_Yes', 'Hx Smoking_No', 'Hx Smoking_Yes', 'Hx Radiothreapy_No', 'Hx Radiothreapy_Yes', 'Thyroid Function_Clinical Hyperthyroidism', 'Thyroid Function_Clinical Hypothyroidism', 'Thyroid Function_Euthyroid', 'Thyroid Function_Subclinical Hyperthyroidism', 'Thyroid Function_Subclinical Hypothyroidism', 'Physical Examination_Diffuse goiter', 'Physical Examination_Multinodular goiter', 'Physical Examination_Normal', 'Physical Examination_Single nodu

### ETAPA 4: Divisão dos dados e encontrando o melhor valor para o KNN

In [167]:
# --------------------------
# 4) ENCONTRANDO MELHOR VALOR DE K
# Dividir em treino+validação (80%) e teste (20%) e dentro do 80% dividir em treino (70% total) e validação (10% total)

moldura("Buscando o melhor valor de k para KNN e dividindo os dados")

X = df_processado.drop('result', axis=1)
y = df_processado['result']

# Primeiro split: separar teste 20%
X_rem, X_test, y_rem, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
# Agora dividir X_rem (80%) em train(70%) e val(10%) == proporção em relação ao rem: train_size = 0.875
X_train, X_val, y_train, y_val = train_test_split(X_rem, y_rem, train_size=0.875, stratify=y_rem, random_state=42)

add_linha("Divisão dos dados")
print('  Train:', X_train.shape, ' Val:', X_val.shape, ' Test:', X_test.shape)

# Normalizar (fit no train)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Buscar k em 1..30
from tqdm import trange
k_range = range(1, min(31, max(2, int(np.sqrt(X_train.shape[0]) * 5))))  # limite razoável
val_scores = []
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    y_pred = knn.predict(X_val_scaled)
    acc = accuracy_score(y_val, y_pred)
    val_scores.append((k, acc))

val_df = pd.DataFrame(val_scores, columns=['k', 'val_accuracy']).sort_values(['val_accuracy','k'], ascending=[False, True])
add_linha("Resultados (validação) — top 10 ks")
print(val_df.head(10).to_string(index=False))

# Melhor k encontrado
best_k = int(val_df.iloc[0]['k'])

# Se o melhor k for 1, pegar o segundo melhor (se existir)
if best_k == 1 and len(val_df) > 1:
    print(f"\n[AVISO] Melhor k encontrado foi 1 (pode causar overfitting). Usando o segundo melhor valor.")
    best_k = int(val_df.iloc[1]['k'])

print(f"\nMelhor k definido para o KNN: {best_k}")

+------------------------------------------------------------+
| Buscando o melhor valor de k para KNN e dividindo os dados |
+------------------------------------------------------------+

# ---------------------------------- Divisão dos dados ----------------------------------- #

  Train: (267, 55)  Val: (39, 55)  Test: (77, 55)

# -------------------------- Resultados (validação) — top 10 ks -------------------------- #

 k  val_accuracy
 1      0.974359
 3      0.974359
 4      0.948718
 5      0.948718
 6      0.948718
 7      0.948718
 8      0.948718
 9      0.948718
10      0.948718
11      0.948718

[AVISO] Melhor k encontrado foi 1 (pode causar overfitting). Usando o segundo melhor valor.

Melhor k definido para o KNN: 3


### ETAPA 5: Hold-Out

In [168]:
# 5) HOLD-OUT (70/30) repetido 50x

moldura("HOLD-OUT (70/30) repetido 50x")

n_repeats = 50
sss = StratifiedShuffleSplit(n_splits=n_repeats, test_size=0.3, random_state=42)
metrics_hold = {'accuracy': [], 'f1': [], 'precision': [], 'recall': [], 'kappa': []}

for train_idx, test_idx in sss.split(X, y):
    X_tr, X_te = X.iloc[train_idx], X.iloc[test_idx]
    y_tr, y_te = y.iloc[train_idx], y.iloc[test_idx]

    sc = MinMaxScaler()
    X_tr_s = sc.fit_transform(X_tr)
    X_te_s = sc.transform(X_te)

    clf = KNeighborsClassifier(n_neighbors=best_k, weights='distance')
    clf.fit(X_tr_s, y_tr)
    y_pred = clf.predict(X_te_s)

    metrics_hold['accuracy'].append(accuracy_score(y_te, y_pred))
    metrics_hold['f1'].append(f1_score(y_te, y_pred, average='macro', zero_division=0))
    metrics_hold['precision'].append(precision_score(y_te, y_pred, average='macro', zero_division=0))
    metrics_hold['recall'].append(recall_score(y_te, y_pred, average='macro', zero_division=0))
    metrics_hold['kappa'].append(cohen_kappa_score(y_te, y_pred))

hold_summary = {m: (np.mean(vals), np.std(vals)) for m, vals in metrics_hold.items()}

add_linha("Sumário do Hold-out (50 rep)")
for m, (mu, sd) in hold_summary.items():
    print(f"{m:<20} {mu:.4f} ± {sd:.4f}")

+-------------------------------+
| HOLD-OUT (70/30) repetido 50x |
+-------------------------------+

# ----------------------------- Sumário do Hold-out (50 rep) ----------------------------- #

accuracy             0.9242 ± 0.0187
f1                   0.9023 ± 0.0250
precision            0.9189 ± 0.0251
recall               0.8904 ± 0.0286
kappa                0.8049 ± 0.0497


### ETAPA 6: K-Fold

In [169]:
# 6) K-FOLD (5)

moldura("K-FOLD (5)")

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
metrics_kfold = {m: [] for m in metrics_hold}

for train_idx, test_idx in kf.split(X, y):
    X_tr, X_te = X.iloc[train_idx], X.iloc[test_idx]
    y_tr, y_te = y.iloc[train_idx], y.iloc[test_idx]

    sc = MinMaxScaler()
    X_tr_s = sc.fit_transform(X_tr)
    X_te_s = sc.transform(X_te)

    clf = KNeighborsClassifier(n_neighbors=best_k, weights='distance')
    clf.fit(X_tr_s, y_tr)
    y_pred = clf.predict(X_te_s)

    metrics_kfold['accuracy'].append(accuracy_score(y_te, y_pred))
    metrics_kfold['f1'].append(f1_score(y_te, y_pred, average='macro', zero_division=0))
    metrics_kfold['precision'].append(precision_score(y_te, y_pred, average='macro', zero_division=0))
    metrics_kfold['recall'].append(recall_score(y_te, y_pred, average='macro', zero_division=0))
    metrics_kfold['kappa'].append(cohen_kappa_score(y_te, y_pred))

kfold_summary = {m: (np.mean(vals), np.std(vals)) for m, vals in metrics_kfold.items()}

add_linha("Sumário do K-Fold (5)")
for m, (mu, sd) in kfold_summary.items():
    print(f"{m:<20} {mu:.4f} ± {sd:.4f}")

+------------+
| K-FOLD (5) |
+------------+

# -------------------------------- Sumário do K-Fold (5) --------------------------------- #

accuracy             0.9242 ± 0.0155
f1                   0.9022 ± 0.0222
precision            0.9235 ± 0.0230
recall               0.8878 ± 0.0283
kappa                0.8049 ± 0.0441


### ETAPA 7: LOOCV

In [170]:
# 7) Leave-Ove-Out

moldura("Leave-One-Out")

loo = LeaveOneOut()
metrics_loo = []

y_true_all = []
y_pred_all = []

print('\nExecutando Leave-One-Out (pode demorar)...')

iteracao = 0
for train_idx, test_idx in loo.split(X):
    X_tr, X_te = X.iloc[train_idx], X.iloc[test_idx]
    y_tr, y_te = y.iloc[train_idx], y.iloc[test_idx]

    sc = MinMaxScaler()
    X_tr_s = sc.fit_transform(X_tr)
    X_te_s = sc.transform(X_te)

    clf = KNeighborsClassifier(n_neighbors=best_k, weights='distance')
    clf.fit(X_tr_s, y_tr)
    y_pred = clf.predict(X_te_s)

    y_true_all.append(y_te.values[0])
    y_pred_all.append(y_pred[0])

    iteracao += 1
    if iteracao % 500 == 0:
        print(f"  Processadas {iteracao} amostras...")

# Calcular métricas finais
metrics_loo = {
    'accuracy': accuracy_score(y_true_all, y_pred_all),
    'f1': f1_score(y_true_all, y_pred_all, average='macro', zero_division=0),
    'precision': precision_score(y_true_all, y_pred_all, average='macro', zero_division=0),
    'recall': recall_score(y_true_all, y_pred_all, average='macro', zero_division=0),
    'kappa': cohen_kappa_score(y_true_all, y_pred_all)
}

loo_summary = {m: (val, 0.0) for m, val in metrics_loo.items()}
add_linha("Sumário do Leave-One-Out")
for m, (mu, sd) in loo_summary.items():
    print(f"{m:<20} {mu:.4f} ± {sd:.4f}")

+---------------+
| Leave-One-Out |
+---------------+

Executando Leave-One-Out (pode demorar)...

# ------------------------------- Sumário do Leave-One-Out ------------------------------- #

accuracy             0.9243 ± 0.0000
f1                   0.9028 ± 0.0000
precision            0.9214 ± 0.0000
recall               0.8882 ± 0.0000
kappa                0.8059 ± 0.0000


### ETAPA 8: Tabelas comparativas

In [ ]:
# 8) Comparativo entre os 3 métodos em formato de tabela
rows = []
# Hold-out
rows.append({
    'method': 'Hold-out (50 rep)',
    'accuracy_mean': hold_summary['accuracy'][0], 'accuracy_std': hold_summary['accuracy'][1],
    'f1_mean': hold_summary['f1'][0], 'f1_std': hold_summary['f1'][1],
    'precision_mean': hold_summary['precision'][0], 'precision_std': hold_summary['precision'][1],
    'recall_mean': hold_summary['recall'][0], 'recall_std': hold_summary['recall'][1],
    'kappa_mean': hold_summary['kappa'][0], 'kappa_std': hold_summary['kappa'][1]
})

# K-Fold
rows.append({
    'method': 'K-Fold (5)',
    'accuracy_mean': kfold_summary['accuracy'][0], 'accuracy_std': kfold_summary['accuracy'][1],
    'f1_mean': kfold_summary['f1'][0], 'f1_std': kfold_summary['f1'][1],
    'precision_mean': kfold_summary['precision'][0], 'precision_std': kfold_summary['precision'][1],
    'recall_mean': kfold_summary['recall'][0], 'recall_std': kfold_summary['recall'][1],
    'kappa_mean': kfold_summary['kappa'][0], 'kappa_std': kfold_summary['kappa'][1]
})

# LOO
rows.append({
    'method': 'Leave-One-Out',
    'accuracy_mean': loo_summary['accuracy'][0], 'accuracy_std': loo_summary['accuracy'][1],
    'f1_mean': loo_summary['f1'][0], 'f1_std': loo_summary['f1'][1],
    'precision_mean': loo_summary['precision'][0], 'precision_std': loo_summary['precision'][1],
    'recall_mean': loo_summary['recall'][0], 'recall_std': loo_summary['recall'][1],
    'kappa_mean': loo_summary['kappa'][0], 'kappa_std': loo_summary['kappa'][1]
})

summary_df = pd.DataFrame(rows)

# Formatação de saída (mean ± std)
def fmt(mean, sd):
    return f"{mean:.4f} ± {sd:.4f}"

out = pd.DataFrame({
    'Method': summary_df['method'],
    'Accuracy': [fmt(r['accuracy_mean'], r['accuracy_std']) for _, r in summary_df.iterrows()],
    'F1-score': [fmt(r['f1_mean'], r['f1_std']) for _, r in summary_df.iterrows()],
    'Precision': [fmt(r['precision_mean'], r['precision_std']) for _, r in summary_df.iterrows()],
    'Recall': [fmt(r['recall_mean'], r['recall_std']) for _, r in summary_df.iterrows()],
    'Kappa': [fmt(r['kappa_mean'], r['kappa_std']) for _, r in summary_df.iterrows()]
})

moldura("Comparativo entre métodos")
print(out.to_string(index=False, col_space=20))

+---------------------------+
| Comparativo entre métodos |
+---------------------------+
              Method             Accuracy             F1-score            Precision               Recall                Kappa
   Hold-out (50 rep)      0.9242 ± 0.0187      0.9023 ± 0.0250      0.9189 ± 0.0251      0.8904 ± 0.0286      0.8049 ± 0.0497
          K-Fold (5)      0.9242 ± 0.0155      0.9022 ± 0.0222      0.9235 ± 0.0230      0.8878 ± 0.0283      0.8049 ± 0.0441
       Leave-One-Out      0.9243 ± 0.0000      0.9028 ± 0.0000      0.9214 ± 0.0000      0.8882 ± 0.0000      0.8059 ± 0.0000
